In [2]:
!pip install pulp
import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 2230  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 1.0  # Weight for PGC
beta = 1.0  # Weight for TGC

# Randomly generated data
waste_generated = {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot on day day
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                               ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
for d in districts:
    for day in days:
        model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) - 1, f"AbsPos_{d}_{day}"
        model_fixed += abs_neg[(d, day)] >= -(g_fixed[(d, day)] * (1 / waste_generated[d][day]) - 1), f"AbsNeg_{d}_{day}"
        model_fixed += abs_diff[(d, day)] == abs_pos[(d, day)] + abs_neg[(d, day)], f"AbsDiff_{d}_{day}"


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * waste_generated[d][day] /
                    truck_capacity for d in districts for t in range(total_trucks) for k in timeslots for day in days)

def PGC():
    return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)

# Define objective
model_fixed += alpha * TCT(x_fixed) + gamma * PGC() - beta * TGC(g_fixed)

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]
                                                     for t in range(total_trucks) for k in timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
    model_fixed += (2 <=lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) <= 3) , f"TotalPickupFrequency_Fixed_{d}"

# Equity constraint as a soft constraint with tolerance
#for d in districts:
  #   model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * SEI[d] for t in range(total_trucks) for k in timeslots for day in days) <= 1.5 * frequency_required[d], f"EquityConstraint_{d}"

# Truck capacity and availability constraints
model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day] for d in districts for t in range(total_trucks) for day in days for k in timeslots) <= truck_capacity * lp.lpSum(x_fixed[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in days), "TotalTruckCapacity"


#for t in range(total_trucks):
 #   for d in districts:
 #       for k in timeslots:
  #          for day in days:
   #             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
for t in range(total_trucks):
    model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0,1,2,3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve()

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_fixed[(d, t, k, day)]) > 0 :
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, k, day)])}")



Fixed Assignment Model:
Status: Optimal
Objective Value: -347.99999742128057
District 0, Truck 2143, Morning, Day 1: Assignment = 1.0
District 0, Truck 973, Morning, Day 4: Assignment = 1.0
District 0, Truck 827, Evening, Day 5: Assignment = 1.0
District 1, Truck 1485, Evening, Day 0: Assignment = 1.0
District 1, Truck 1991, Evening, Day 4: Assignment = 1.0
District 1, Truck 316, Evening, Day 6: Assignment = 1.0
District 2, Truck 903, Morning, Day 0: Assignment = 1.0
District 2, Truck 2147, Evening, Day 2: Assignment = 1.0
District 2, Truck 1645, Morning, Day 3: Assignment = 1.0
District 3, Truck 1986, Morning, Day 0: Assignment = 1.0
District 3, Truck 1256, Evening, Day 2: Assignment = 1.0
District 3, Truck 1100, Morning, Day 6: Assignment = 1.0
District 4, Truck 979, Evening, Day 4: Assignment = 1.0
District 4, Truck 706, Evening, Day 5: Assignment = 1.0
District 4, Truck 262, Evening, Day 6: Assignment = 1.0
District 5, Truck 333, Evening, Day 0: Assignment = 1.0
District 5, Truck 3